In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/ALBEF

/content/drive/MyDrive/ALBEF


In [41]:
%ls ./data


json_pretrain_data/  __MACOSX/  vqa/  vqa_val.json


In [3]:
!pip install transformers==4.25.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.25.1 which is incompatible.


In [4]:
!pip install ruamel.yaml==0.17.*

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 34.6 MB/s eta 0:00:00


In [5]:
import argparse
import os
import ruamel.yaml as yaml
import numpy as np
import random
import time
import datetime
import json
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
import torch.distributed as dist

from models.model_vqa import ALBEF
from models.vit import interpolate_pos_embed
from models.tokenization_bert import BertTokenizer

import utils
from dataset.utils import save_result
from dataset import create_dataset, create_sampler, create_loader, vqa_collate_fn

from scheduler import create_scheduler
from optim import create_optimizer

/usr/local/lib/python3.11/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
def train(model, data_loader, optimizer, tokenizer, epoch, warmup_steps, device, scheduler, config):
    # train
    model.train()

    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', utils.SmoothedValue(window_size=1, fmt='{value:.6f}'))
    metric_logger.add_meter('loss', utils.SmoothedValue(window_size=1, fmt='{value:.4f}'))

    header = 'Train Epoch: [{}]'.format(epoch)
    print_freq = 50
    step_size = 100
    warmup_iterations = warmup_steps*step_size

    for i,(image, question, answer, weights, n) in enumerate(metric_logger.log_every(data_loader, print_freq, header)):
        image, weights = image.to(device,non_blocking=True), weights.to(device,non_blocking=True)
        question_input = tokenizer(question, padding='longest', truncation=True, max_length=25, return_tensors="pt").to(device)
        answer_input = tokenizer(answer, padding='longest', return_tensors="pt").to(device)

        if epoch>0 or not config['warm_up']:
            alpha = config['alpha']
        else:
            alpha = config['alpha']*min(1,i/len(data_loader))

        loss = model(image, question_input, answer_input, train=True, alpha=alpha, k=n, weights=weights)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        metric_logger.update(loss=loss.item())
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

        if epoch==0 and i%step_size==0 and i<=warmup_iterations:
            scheduler.step(i//step_size)

    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger.global_avg())
    return {k: "{:.3f}".format(meter.global_avg) for k, meter in metric_logger.meters.items()}

In [8]:
@torch.no_grad()
def evaluation(model, data_loader, tokenizer, device, config) :
    # test
    model.eval()

    metric_logger = utils.MetricLogger(delimiter="  ")
    header = 'Generate VQA test result:'
    print_freq = 50

    result = []

    answer_list = [answer+config['eos'] for answer in data_loader.dataset.answer_list]
    answer_input = tokenizer(answer_list, padding='longest', return_tensors='pt').to(device)

    for n, (image, question, question_id) in enumerate(metric_logger.log_every(data_loader, print_freq, header)):
        image = image.to(device,non_blocking=True)
        question_input = tokenizer(question, padding='longest', return_tensors="pt").to(device)

        topk_ids, topk_probs = model(image, question_input, answer_input, train=False, k=config['k_test'])

        for ques_id, topk_id, topk_prob in zip(question_id, topk_ids, topk_probs):
            ques_id = int(ques_id.item())
            _, pred = topk_prob.max(dim=0)
            result.append({"question_id":ques_id, "answer":data_loader.dataset.answer_list[topk_id[pred]]})

    return result

In [9]:

import requests
import os

# Final direct URL (avoid redirects from Google)
url = "https://storage.googleapis.com/sfr-pcl-data-research/ALBEF/ALBEF.pth"
output_path = "ALBEF.pth"

def download_model(url, save_path):
    print(f"Downloading model from {url}")
    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raise error for bad responses

    total = int(response.headers.get('content-length', 0))
    with open(save_path, 'wb') as file, \
            requests.get(url, stream=True) as r:
        for chunk in r.iter_content(chunk_size=8192):
            if chunk:
                file.write(chunk)

    print(f"Model saved to {save_path}")

# download_model(url, output_path)


In [33]:

url = "https://storage.googleapis.com/sfr-pcl-data-research/ALBEF/ALBEF_4M.pth"
output_path = "ALBEF_4M.pth"
# download_model(url, output_path)


In [ ]:

!git config --global user.email "ranran.lei@berkeley.edu"
!git config --global user.name "VickieRanran"

In [ ]:
# !git clone https://github.com/Yan9595/ALBEF.git

Cloning into 'ALBEF'...
remote: Enumerating objects: 458, done.
remote: Counting objects: 100% (250/250), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 458 (delta 150), reused 137 (delta 106), pack-reused 208 (from 1)
Receiving objects: 100% (458/458), 77.17 MiB | 14.90 MiB/s, done.
Resolving deltas: 100% (197/197), done.
Updating files: 100% (165/165), done.


In [ ]:
!git push origin Test -f

error: src refspec Test does not match any
error: failed to push some refs to 'https://github.com/Yan9595/ALBEF.git'


In [10]:
args = argparse.Namespace()
args.config = './configs/VQA.yaml'
args.checkpoint = None
args.output_dir = './output/vqa'
args.evaluate = False
args.text_encoder = 'bert-base-uncased'
args.text_decoder = 'bert-base-uncased'
args.device = 'cuda'
args.seed = 42
args.distributed = True

config = yaml.load(open(args.config, 'r'), Loader=yaml.Loader)

In [11]:
args.result_dir = os.path.join(args.output_dir, 'result')

Path(args.output_dir).mkdir(parents=True, exist_ok=True)
Path(args.result_dir).mkdir(parents=True, exist_ok=True)

yaml.dump(config, open(os.path.join(args.output_dir, 'config.yaml'), 'w'))

In [49]:
print(len(train_loader))

0


In [54]:

datasets[0]

In [46]:
utils.init_distributed_mode(args)

device = torch.device(args.device)

# fix the seed for reproducibility
seed = args.seed + utils.get_rank()
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
cudnn.benchmark = True

start_epoch = 0
max_epoch = config['schedular']['epochs']
warmup_steps = config['schedular']['warmup_epochs']


#### Dataset ####
print("Creating vqa datasets")
datasets = create_dataset('vqa', config)

if args.distributed:
    num_tasks = utils.get_world_size()
    global_rank = utils.get_rank()
    samplers = create_sampler(datasets, [True, False], num_tasks, global_rank)
else:
    samplers = [None, None]

train_loader, test_loader = create_loader(datasets,samplers,
                                          batch_size=[config['batch_size_train'],config['batch_size_test']],
                                          num_workers=[4,4],is_trains=[True, False],
                                          collate_fns=[vqa_collate_fn,None])

tokenizer = BertTokenizer.from_pretrained(args.text_encoder)

#### Model ####
print("Creating model")
model = ALBEF(config=config, text_encoder=args.text_encoder, text_decoder=args.text_decoder, tokenizer=tokenizer)
model = model.to(device)

arg_opt = utils.AttrDict(config['optimizer'])
optimizer = create_optimizer(arg_opt, model)
arg_sche = utils.AttrDict(config['schedular'])
lr_scheduler, _ = create_scheduler(arg_sche, optimizer)

if args.checkpoint:
    checkpoint = torch.load(args.checkpoint, map_location='cpu')
    if args.evaluate:
        state_dict = checkpoint
    else:
        state_dict = checkpoint['model']

    # reshape positional embedding to accomodate for image resolution change
    pos_embed_reshaped = interpolate_pos_embed(state_dict['visual_encoder.pos_embed'],model.visual_encoder)
    state_dict['visual_encoder.pos_embed'] = pos_embed_reshaped

    if not args.evaluate:
        if config['distill']:
            m_pos_embed_reshaped = interpolate_pos_embed(state_dict['visual_encoder_m.pos_embed'],model.visual_encoder_m)
            state_dict['visual_encoder_m.pos_embed'] = m_pos_embed_reshaped

        for key in list(state_dict.keys()):
            #print(state_dict.keys())
            if 'bert' in key:
                encoder_key = key.replace('bert.','')
                state_dict[encoder_key] = state_dict[key]
            # intialize text decoder as multimodal encoder (last 6 layers of model.text_encoder)
            if 'text_encoder' in key:
                if 'layer' in key:
                    encoder_keys = key.split('.')
                    layer_num = int(encoder_keys[4])
                    if layer_num<6:
                        del state_dict[key]
                        continue
                    else:
                        decoder_layer_num = (layer_num-6)
                        encoder_keys[4] = str(decoder_layer_num)
                        encoder_key = '.'.join(encoder_keys)
                else:
                    encoder_key = key
                decoder_key = encoder_key.replace('text_encoder','text_decoder')
                state_dict[decoder_key] = state_dict[key]

                del state_dict[key]

    msg = model.load_state_dict(state_dict,strict=False)
    print('load checkpoint from %s'%args.checkpoint)
    print(msg)


model_without_ddp = model
if args.distributed:
    model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.gpu])
    model_without_ddp = model.module


print("Start training")
start_time = time.time()

for epoch in range(start_epoch, max_epoch):
    if epoch>0:
        lr_scheduler.step(epoch+warmup_steps)

    if not args.evaluate:
        if args.distributed:
            train_loader.sampler.set_epoch(epoch)

        train_stats = train(model, train_loader, optimizer, tokenizer, epoch, warmup_steps, device, lr_scheduler, config)

    if args.evaluate:
        break

    if utils.is_main_process():
        log_stats = {**{f'train_{k}': v for k, v in train_stats.items()},
                      'epoch': epoch,
                    }
        with open(os.path.join(args.output_dir, "log.txt"),"a") as f:
            f.write(json.dumps(log_stats) + "\n")

        save_obj = {
            'model': model_without_ddp.state_dict(),
            'optimizer': optimizer.state_dict(),
            'lr_scheduler': lr_scheduler.state_dict(),
            'config': config,
            'epoch': epoch,
        }
        torch.save(save_obj, os.path.join(args.output_dir, 'checkpoint_%02d.pth'%epoch))

    dist.barrier()

vqa_result = evaluation(model, test_loader, tokenizer, device, config)
result_file = save_result(vqa_result, args.result_dir, 'vqa_result_epoch%d'%epoch)

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))

Not using distributed mode
Creating vqa datasets
Creating model
Start training


ZeroDivisionError: float division by zero

In [13]:
import os
import urllib.request
import zipfile

# Create base directory
base_dir = "data/vqa"
os.makedirs(base_dir, exist_ok=True)

# List of (URL, local filename)
vqa_files = [
    # Questions
    ("https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Questions_Train_mscoco.zip", "vqa_train_question.zip"),
    ("https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Questions_Val_mscoco.zip", "vqa_val_question.zip"),
    ("https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Questions_Test_mscoco.zip", "vqa_test_question.zip"),

    # Annotations
    ("https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Annotations_Train_mscoco.zip", "v2_Annotations_Train_mscoco.zip"),
    ("https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Annotations_Val_mscoco.zip", "v2_Annotations_Val_mscoco.zip"),

    # # Images
    # ("http://images.cocodataset.org/zips/train2014.zip", "train2014.zip"),
    # ("http://images.cocodataset.org/zips/val2014.zip", "val2014.zip"),
    # ("http://images.cocodataset.org/zips/test2015.zip", "test2015.zip"),
]

def download_and_extract(url, dest_path):
    print(f"Downloading from {url}...")
    urllib.request.urlretrieve(url, dest_path)
    print(f"Extracting {dest_path}...")
    with zipfile.ZipFile(dest_path, 'r') as zip_ref:
        zip_ref.extractall(base_dir)
    os.remove(dest_path)
    print(f"Completed: {dest_path}")

for url, filename in vqa_files:
    try:
        download_and_extract(url, os.path.join(base_dir, filename))
    except Exception as e:
        print(f"Error processing {url}: {e}")


Extracting data/vqa/vqa_train_question.zip...
Completed: data/vqa/vqa_train_question.zip
Extracting data/vqa/vqa_val_question.zip...
Completed: data/vqa/vqa_val_question.zip
Extracting data/vqa/vqa_test_question.zip...
Completed: data/vqa/vqa_test_question.zip
Extracting data/vqa/v2_Annotations_Train_mscoco.zip...
Completed: data/vqa/v2_Annotations_Train_mscoco.zip
Extracting data/vqa/v2_Annotations_Val_mscoco.zip...
Completed: data/vqa/v2_Annotations_Val_mscoco.zip


In [ ]:
!cp -r output drive/MyDrive/ALBEF/

In [ ]:
import os
import requests
import zipfile

# Define function to download and extract
def download_and_extract(url, zip_path, extract_dir):
    os.makedirs(os.path.dirname(zip_path), exist_ok=True)

    # Step 1: Download the zip file
    print(f"Downloading from {url}...")
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(zip_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    print(f"Saved to {zip_path}")

    # Step 2: Extract the zip
    print(f"Extracting to {extract_dir}...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print("Extraction complete.")

    # Optional cleanup
    os.remove(zip_path)
    print("Cleaned up zip file.")

# Define paths and URL
url = "https://storage.googleapis.com/sfr-pcl-data-research/ALBEF/json_pretrain.zip"
zip_path = "data/json_pretrain.zip"
extract_dir = "json_pretrain_data"

# Download and extract
download_and_extract(url, zip_path, extract_dir)


Saved to data/json_pretrain.zip
Extracting to json_pretrain_data...
Extraction complete.
Cleaned up zip file.


In [ ]:
%ls ./data/json_pre

ALBEF.ipynb         examples/            output/           scheduler/
ALBEF.pth           Grounding.py         predict.py        SECURITY.md
CODE_OF_CONDUCT.md  img.png              Pretrain_nlvr.py  utils.py
CODEOWNERS          json_pretrain_data/  Pretrain.py       VE.py
cog.yaml            LICENSE.txt          __pycache__/      visualization.ipynb
configs/            models/              README.md         VQA.py
data/               NLVR.py              refTools/         vqaTools/
dataset/            optim/               Retrieval.py
